In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

# Load the original CSV file
input_csv_path = "gb_input.csv"  # Replace with your input CSV file path
output_csv_path = "gb_doutput.csv"  # Replace with the desired output CSV file path

# Initialize a Selenium webdriver
driver = webdriver.Chrome()

# Read the CSV file into a DataFrame
#df = pd.read_csv(input_csv_path)

# Read the CSV file into a DataFrame with the appropriate encoding
df = pd.read_csv(input_csv_path, encoding='latin-1')  # Try 'latin-1' or 'ISO-8859-1' if needed


# Initialize lists to store results
ids = []
priority_dates = []
cite_numbers = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    result_link = row['result_link']
    
    # Open the URL in the browser
    driver.get(result_link)
    
    # Get the page source after JavaScript rendering
    page_source = driver.page_source
    
    # Parse the page source using Beautiful Soup
    soup = BeautifulSoup(page_source, "html.parser")
    
    # Find the "Cited by" element using the HTML structure
    cited_by_element = soup.find("h3", {"id": "citedBy"})
    
    # Extract the "Cited by" count if found
    if cited_by_element:
        cited_by_text = cited_by_element.get_text()
        cited_by_count = int(re.search(r"\d+", cited_by_text).group())
        ids.append(row['id'])
        priority_dates.append(row['priority_date'])
        cite_numbers.append(cited_by_count)
    else:
        ids.append(row['id'])
        priority_dates.append(row['priority_date'])
        cite_numbers.append(None)

# Close the browser
driver.quit()

# Create a new DataFrame with the results
result_df = pd.DataFrame({'id': ids, 'priority_date': priority_dates, 'cite_number': cite_numbers})

# Save the results to a new CSV file
result_df.to_csv(output_csv_path, index=False)
